## Installing Pytorch

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x5742e000 @  0x7fa47ff752a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


## Building Fairseq

In [2]:
!git clone https://github.com/pytorch/fairseq.git

Cloning into 'fairseq'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 2814 (delta 63), reused 74 (delta 46), pack-reused 2698
Receiving objects: 100% (2814/2814), 3.02 MiB | 17.97 MiB/s, done.
Resolving deltas: 100% (2028/2028), done.


In [0]:
import os
os.chdir("fairseq/")

In [4]:
!pip install -r requirements.txt
%run -i 'setup.py' build develop

    100% |████████████████████████████████| 430kB 6.6MB/s 
    100% |████████████████████████████████| 163kB 6.9MB/s 
  Running setup.py bdist_wheel for pycparser ... - \ done
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built pycparser
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/scripts
copying scripts/read_binarized.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/average_checkpoints.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/__init__.py -> build/lib.linux-x86_64-3.6/scripts
copying scripts/build_sym_alignment.py -> build/lib.linux-x86_64-3.6/scripts
creating build/lib.linux-x86_64-3.6/tests
copying tests/test_convtbc.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_average_checkpoints.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_binaries.py -> build/lib.linux-x86_64-3.6/tests
copying tests

## Model for Setswana


### Data Preprop
Upload the data in the form train.en, train.tn, valid.en, valid.tn, test.en, test.tn.

In [0]:
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang tn --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/entn

Namespace(alignfile=None, destdir='data-bin/entn', joined_dictionary=False, nwordssrc=-1, nwordstgt=-1, only_source=False, output_format='binary', padding_factor=8, source_lang='en', srcdict=None, target_lang='tn', testpref='../test', tgtdict=None, thresholdsrc=0, thresholdtgt=0, trainpref='../train', validpref='../valid', workers=1)
| [en] Dictionary: 43511 types
| [en] ../train.en: 111300 sents, 1816310 tokens, 0.0% replaced by <unk>
| [en] Dictionary: 43511 types
| [en] ../valid.en: 44700 sents, 599799 tokens, 1.03% replaced by <unk>
| [en] Dictionary: 43511 types
| [en] ../test.en: 3000 sents, 56851 tokens, 0.997% replaced by <unk>
| [tn] Dictionary: 56535 types
| [tn] ../train.tn: 111300 sents, 2259026 tokens, 0.0% replaced by <unk>
| [tn] Dictionary: 56535 types
| [tn] ../valid.tn: 44700 sents, 703248 tokens, 1.47% replaced by <unk>
| [tn] Dictionary: 56535 types
| [tn] ../test.tn: 3000 sents, 69838 tokens, 1.43% replaced by <unk>
| Wrote preprocessed data to data-bin/entn


### Train Model

In [0]:
!mkdir -p checkp/fconv
!python train.py data-bin/entn \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir chekp/fconv

Namespace(arch='fconv', bucket_cap_mb=150, clip_norm=0.1, criterion='cross_entropy', data=['data-bin/entn'], ddp_backend='c10d', decoder_attention='True', decoder_embed_dim=512, decoder_embed_path=None, decoder_layers='[(512, 3)] * 20', decoder_out_embed_dim=256, device_id=0, distributed_backend='nccl', distributed_init_method=None, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.2, encoder_embed_dim=512, encoder_embed_path=None, encoder_layers='[(512, 3)] * 20', fix_batches_to_gpus=False, fp16=False, fp16_init_scale=128, keep_interval_updates=-1, left_pad_source='True', left_pad_target='False', log_format=None, log_interval=1000, lr=[0.25], lr_scheduler='reduce_lr_on_plateau', lr_shrink=0.1, max_epoch=0, max_sentences=None, max_sentences_valid=None, max_source_positions=1024, max_target_positions=1024, max_tokens=4000, max_update=0, min_loss_scale=0.0001, min_lr=1e-05, momentum=0.99, no_epoch_checkpoints=False, no_progress_bar=False, no_save=False, optimiz

### Testing
Generate translations from the test data. Calculate the Bleu score.

In [0]:
#!python generate.py data-bin/entn \
#    --path chkp/fconv/checkpoint_best.pt \
#    --batch-size 128 --beam 5
output = %run 'generate.py' data-bin/entn --path chekp/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang tn

Namespace(beam=5, cpu=False, data=['data-bin/entn'], diverse_beam_groups=1, diverse_beam_strength=0.5, fp16=False, fp16_init_scale=128, gen_subset='test', left_pad_source='True', left_pad_target='False', lenpen=1, log_format=None, log_interval=1000, max_len_a=0, max_len_b=200, max_sentences=128, max_source_positions=1024, max_target_positions=1024, max_tokens=None, min_len=1, model_overrides='{}', nbest=1, no_beamable_mm=False, no_early_stop=False, no_progress_bar=False, num_shards=1, path='chekp/fconv/checkpoint_best.pt', prefix_size=0, print_alignment=False, quiet=False, raw_text=False, remove_bpe='@@ ', replace_unk=None, sampling=False, sampling_temperature=1, sampling_topk=-1, score_reference=False, seed=1, shard_id=0, skip_invalid_size_inputs_valid_test=False, source_lang='en', target_lang='tn', task='translation', unkpen=0, unnormalized=False, upsample_primary=1)
| [en] dictionary: 43512 types
| [tn] dictionary: 56536 types
| data-bin/entn test 3000 examples
| ['data-bin/entn'] t

This step is unnecessary as the BLEU score is calculated at the end of generation.

In [0]:
#I ran this locally
#grep ^H /tmp/gen.out | cut -f3- > /tmp/gen.out.sys
#grep ^T /tmp/gen.out | cut -f2- > /tmp/gen.out.ref

%run 'score.py' --sys ../gen.out.sys --ref ../gen.out.ref

Namespace(ignore_case=False, order=4, ref='../gen.out.ref', sys='../gen.out.sys')
BLEU4 = 25.06, 54.2/30.3/19.4/13.0 (BP=0.987, ratio=0.987, syslen=65962, reflen=66838)


## Model for isiZulu
### Data Preprop
Upload the data in the form train.en, train.tn, valid.en, valid.tn, test.en, test.tn.


In [5]:
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang zu --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enzu

Namespace(alignfile=None, destdir='data-bin/enzu', joined_dictionary=False, nwordssrc=-1, nwordstgt=-1, only_source=False, output_format='binary', padding_factor=8, source_lang='en', srcdict=None, target_lang='zu', testpref='../test', tgtdict=None, thresholdsrc=0, thresholdtgt=0, trainpref='../train', validpref='../valid', workers=1)
| [en] Dictionary: 27223 types
| [en] ../train.en: 24839 sents, 383259 tokens, 0.0% replaced by <unk>
| [en] Dictionary: 27223 types
| [en] ../valid.en: 7646 sents, 176203 tokens, 3.26% replaced by <unk>
| [en] Dictionary: 27223 types
| [en] ../test.en: 3000 sents, 51261 tokens, 8.08% replaced by <unk>
| [zu] Dictionary: 68343 types
| [zu] ../train.zu: 24839 sents, 299880 tokens, 0.0% replaced by <unk>
| [zu] Dictionary: 68343 types
| [zu] ../valid.zu: 7646 sents, 128596 tokens, 21.0% replaced by <unk>
| [zu] Dictionary: 68343 types
| [zu] ../test.zu: 3000 sents, 38339 tokens, 25.9% replaced by <unk>
| Wrote preprocessed data to data-bin/enzu


### Train Model

In [10]:
!mkdir -p checkp/fconv
!python train.py data-bin/enzu \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 2000 \
    --arch fconv_wmt_en_de --save-dir checkp/fconv

Namespace(arch='fconv_wmt_en_de', bucket_cap_mb=150, clip_norm=0.1, criterion='cross_entropy', data=['data-bin/enzu'], ddp_backend='c10d', decoder_attention='True', decoder_embed_dim=768, decoder_embed_path=None, decoder_layers='[(512, 3)] * 9 + [(1024, 3)] * 4 + [(2048, 1)] * 2', decoder_out_embed_dim=512, device_id=0, distributed_backend='nccl', distributed_init_method=None, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.2, encoder_embed_dim=768, encoder_embed_path=None, encoder_layers='[(512, 3)] * 9 + [(1024, 3)] * 4 + [(2048, 1)] * 2', fix_batches_to_gpus=False, fp16=False, fp16_init_scale=128, keep_interval_updates=-1, left_pad_source='True', left_pad_target='False', log_format=None, log_interval=1000, lr=[0.25], lr_scheduler='reduce_lr_on_plateau', lr_shrink=0.1, max_epoch=0, max_sentences=None, max_sentences_valid=None, max_source_positions=1024, max_target_positions=1024, max_tokens=2000, max_update=0, min_loss_scale=0.0001, min_lr=1e-05, momentum

### Testing
Generate translations from the test data. Calculate the Bleu score.

In [13]:
#!python generate.py data-bin/entn \
#    --path chkp/fconv/checkpoint_best.pt \
#    --batch-size 128 --beam 5
output = %run 'generate.py' data-bin/enzu --path checkp/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang zu

Namespace(beam=5, cpu=False, data=['data-bin/enzu'], diverse_beam_groups=1, diverse_beam_strength=0.5, fp16=False, fp16_init_scale=128, gen_subset='test', left_pad_source='True', left_pad_target='False', lenpen=1, log_format=None, log_interval=1000, max_len_a=0, max_len_b=200, max_sentences=128, max_source_positions=1024, max_target_positions=1024, max_tokens=None, min_len=1, model_overrides='{}', nbest=1, no_beamable_mm=False, no_early_stop=False, no_progress_bar=False, num_shards=1, path='checkp/fconv/checkpoint_best.pt', prefix_size=0, print_alignment=False, quiet=False, raw_text=False, remove_bpe='@@ ', replace_unk=None, sampling=False, sampling_temperature=1, sampling_topk=-1, score_reference=False, seed=1, shard_id=0, skip_invalid_size_inputs_valid_test=False, source_lang='en', target_lang='zu', task='translation', unkpen=0, unnormalized=False, upsample_primary=1)
| [en] dictionary: 27224 types
| [zu] dictionary: 68344 types
| data-bin/enzu test 3000 examples
| ['data-bin/enzu'] 